# Generator
- An iterator is an object representing a stream of data; this object returns the data one element at a time
- Generators are a special class of functions that simplify the task of writing iterators
- Regular functions compute a value and return it, but generators return an iterator that returns a stream of values
- Any function containing a ```yield``` keyword is a generator function
- When calling a generator function, it doesn’t return a single value; instead it returns a generator object that supports the iterator protocol
- On executing the ```yield``` expression, the generator outputs the value, similar to a return statement
- The big difference between ```yield and a return``` statement is that on reaching a ```yield``` the generator’s state of execution is suspended and local variables are preserved
- On the next call to the ```generator’s next()``` method, the function will resume executing
- ```yield``` pauses a function. ```next()``` resumes where it left off

In [1]:
def fibonacci(max):
    a, b = 0, 1
    while a < max:
        yield a
        a, b = b, a+b

f = fibonacci(100)

In [2]:
f

<generator object fibonacci at 0x000001B84401A0B0>

In [3]:
print([x for x in f])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


- Behind the scenes, the for statement calls `iter()` on the container object. The function returns an iterator object that defines the method `__next__()` which accesses elements in the container one at a time
- When there are no more elements, `__next__()` raises a StopIteration exception which tells the for loop to terminate

In [5]:
f.__next__()

StopIteration: 

In [6]:
next(f)

StopIteration: 

In [4]:
f = fibonacci(200)
print(dir(f))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


- Add iterator behavior to a class by defining an `__iter__()` method which returns an object with a `__next__()` method. If the class defines `__next__()`, then `__iter__()` can just return self

In [14]:
class fibonacci_class():
    def __init__(self, num):
        self.num = num
    def __iter__(self):
        self.a, self.b = 0, 1
        return self
    def __next__(self):
        if self.a > self.num:
            raise StopIteration
        fib = self.a
        self.a, self.b = self.b, self.a + self.b
        return fib

f = fibonacci_class(100)
f

In [15]:
print([x for x in f])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


In [18]:
iter_f = iter(f)
print(next(iter_f))
print(next(iter_f))
print(next(iter_f))
print(next(iter_f))
print(f==iter_f)

0
1
1
2
True


In [19]:
class fibonacci_class():
    def __init__(self, num):
        self.a, self.b = 0, 1
        self.num = num
    def __iter__(self):
        # self.a, self.b = 0, 1
        return self
    def __next__(self):
        if self.a > self.num:
            raise StopIteration
        fib = self.a
        self.a, self.b = self.b, self.a + self.b
        return fib

In [20]:
f = fibonacci_class(100)
print([x for x in f])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]


In [23]:
iter_f = iter(f)
print(f==iter_f)
print(next(iter_f))

True


StopIteration: 

# Coroutine
- A coroutine is syntactically like a generator
- In a coroutine,  `yield` usually appears on the `right side of an expression`. And it may or may not produce a value (If there is no expression after the `yield` keyword, the generator `yields None`
- The coroutine may receive data from the caller which uses `send(data)` instead of `next() function (or __next__() method`
- The caller usually pulls data from the generator & pushes data into the coroutines
- It is even possible that no data goes in or out through the `yield` keyword.
- Regardless of the flow of data, `yield` is a control flow device that can be used to implement cooperative multi-tasking: each coroutine yields control to a central scheduler so that other coroutines can be activated 

In [39]:
def grep(text):
    print(f'Looking for pattern: {text}')
    while True:
        line = (yield)
        if text in line:
            print(line)

g = grep('python')
g

<generator object grep at 0x0000017D0AB64740>

In [40]:
g.send('python is awesome')

TypeError: can't send non-None value to a just-started generator

- All coroutines must be "primed" by first calling `__next__()` or `send(None)`

In [42]:
g.send(None)

Looking for pattern: python


In [43]:
g.send('python is awesome')

python is awesome


### Deccorate a coroutine

In [33]:
from functools import wraps

def coroutine(orig_func):
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        cr_orig_func = orig_func(*args, **kwargs)
        cr_orig_func.send(None)
        return cr_orig_func
    return wrapper

@coroutine
def grep(text):
    print(f'Looking for pattern: {text}')
    while True:
        line = (yield)
        if text in line:
            print(line)

g = grep('python')

Looking for pattern: python


- `@coroutine` decorator calls `__next__()` or `send(None)` to activate a coroutine

In [34]:
g.send('python is awesome')

python is awesome


In [45]:
def gen_fn():
    received_1 = (yield 1)
    print(received_1)
    
    received_2 = (yield 2)
    print(received_2)
    
    return 'End of gen_fn()'

In [47]:
g = gen_fn()
try:
    g.send(None)
    gen_fn_ret1 = g.send('sent Hello to received_1')
    gen_fn_ret2 = g.send('sent Bye to received_2')
    gen_fn_ret3 = g.send('Bye')
except StopIteration as e:
    print(e)

sent Hello to received_1
sent Bye to received_2
End of gen_fn()


In [52]:
g = gen_fn()
try:
    gen_fn_ret1 = g.send(None)
    print(f'gen_fn_ret1: {gen_fn_ret1}')
    
    gen_fn_ret2 = g.send('sent Hello to received_1')
    print(f'gen_fn_ret2: {gen_fn_ret2}')
    
    gen_fn_ret3 = g.send('sent Bye to received_2')
    print(f'gen_fn_ret3: {gen_fn_ret3}')
except StopIteration as e:
    print(e)

gen_fn_ret1: 1
sent Hello to received_1
gen_fn_ret2: 2
sent Bye to received_2
End of gen_fn()


- Three phases in the execution of the  `gen_fn()`.
- Each phase ends in a `yield` expression, and the next phase starts in the same line when the value of the `yield` expression is assigned to a variable

- Coroutines is a generator that can consume data
- In the beginning, the last instruction pointer is -1, meaning the generator has not begun

In [61]:
g = gen_fn()
print(g)
print(g.gi_frame.f_lasti)

<generator object gen_fn at 0x0000017D0AB6A890>
-1


- The generator's instruction pointer is now 2 bytecodes from the start, part way through the 32 bytes of compiled Python. Because the generator is suspending from the `(yield 1)` expression. Its stack frame doesn’t have local variables


In [62]:
print(g.send(None))
print(g.gi_frame.f_lasti)
print(g.gi_frame.f_locals)
print(len(g.gi_code.co_code))

1
2
{}
32


- The generator's instruction pointer is now 16 bytecodes from the start. Its stack frame now contains the local variable `received_1`

In [63]:
print(g.send('sent Hello to received_1'))
print(g.gi_frame.f_lasti)
print(g.gi_frame.f_locals)
print(len(g.gi_code.co_code))

sent Hello to received_1
2
16
{'received_1': 'sent Hello to received_1'}
32


- When call send again, the generator continues from its second `yield` and finishes by raising special `StopIteration` exception. The exception has value, which is the return value of the generator

In [64]:
print(g.send('sent Bye to received_2'))

sent Bye to received_2


StopIteration: End of gen_fn()

- A coroutine can delegate work to a sub-coroutine with `yield from` and receive the result of the work

In [66]:
def gen_fn():
    received_1 = (yield 1)
    print(received_1)
    
    received_2 = (yield 2)
    print(received_2)
    
    return 'End of gen_fn()'

def call_fn():
    g = gen_fn()
    g_return = yield from g
    print(f'return value of yield-from:  {g_return}')
    return 'return from call_fn()'

caller = call_fn()
try:
    r1 = caller.send(None)
    print('r1: ',r1)
    r2 = caller.send('Hello')
    print('r2: ',r2)
    r3 = caller.send('Bye') # raise StopIteration here
    print('r3: ',r3)
except StopIteration as e:
    print(e)

r1:  1
Hello
r2:  2
Bye
return value of yield-from:  End of gen_fn()
return from call_fn()
